<a href="https://colab.research.google.com/github/AlenaPotato/pet_projects/blob/main/SQL_projects/sql_test_sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Мотивация к проекту
---
Мне было предложено решить тестовое задание на знание SQL. Причём задание было не на специальной тренировочной платформе, а в виде excel-файла с таблицами "было" и "что нужно получить".

Поскольку у меня не было предустановленного приложения для управления базами данных, я решила создать свою в colab и выполнить задание тут.

# Setup

In [1]:
import sqlite3
conn = sqlite3.connect('test.db')

In [2]:
conn.execute(''' CREATE TABLE sales(
                  date DATETIME,
                  product NVARCHAR,
                  sales INTEGER); ''')
conn.commit()

print('The table sales is created.')

The table sales is created.


In [3]:
conn.execute('''
INSERT INTO
    sales (date, product, sales)
VALUES
    ('2022-01-01','a',1979),
    ('2022-02-01','a',6416),
    ('2022-03-01','a',7332),
    ('2022-04-01','a',3446),
    ('2022-05-01','a',5276),
    ('2022-06-01','a',5815),
    ('2022-07-01','a',1070),
    ('2022-08-01','a',8589),
    ('2022-09-01','a',1794),
    ('2022-10-01','a',9399),
    ('2022-11-01','a',3988),
    ('2022-12-01','a',2076),
    ('2023-01-01','a',9460),
    ('2023-02-01','a',0),
    ('2023-03-01','a',4915),
    ('2023-04-01','a',8284),
    ('2023-05-01','a',5621),
    ('2023-06-01','a',2764),
    ('2023-07-01','a',3158),
    ('2023-08-01','a',2453),
    ('2023-09-01','a',5182),
    ('2022-01-01','b',4195),
    ('2022-02-01','b',9905),
    ('2022-03-01','b',9628),
    ('2022-04-01','b',4253),
    ('2022-05-01','b',1376),
    ('2022-06-01','b',9605),
    ('2022-07-01','b',9920),
    ('2022-08-01','b',9646),
    ('2022-09-01','b',9730),
    ('2022-10-01','b',2219),
    ('2022-11-01','b',4498),
    ('2022-12-01','b',9784),
    ('2023-01-01','b',1889),
    ('2023-02-01','b',8906),
    ('2023-03-01','b',4355),
    ('2023-04-01','b',4695),
    ('2023-05-01','b',2488),
    ('2023-06-01','b',1724),
    ('2023-07-01','b',9436),
    ('2023-08-01','b',7706),
    ('2023-09-01','b',3363),
    ('2022-01-01','c',9879),
    ('2022-02-01','c',1567),
    ('2022-03-01','c',1783),
    ('2022-04-01','c',8777),
    ('2022-05-01','c',6213),
    ('2022-07-01','c',1622),
    ('2022-08-01','c',3691),
    ('2022-09-01','c',7216),
    ('2022-10-01','c',9077),
    ('2022-11-01','c',3979),
    ('2022-12-01','c',7714),
    ('2023-01-01','c',5837),
    ('2023-02-01','c',9410),
    ('2023-03-01','c',2200),
    ('2023-04-01','c',7888),
    ('2023-05-01','c',7998),
    ('2023-06-01','c',8165),
    ('2023-07-01','c',2699),
    ('2023-08-01','c',7454),
    ('2023-09-01','c',1268);
''')

conn.commit()

In [4]:
conn = sqlite3.connect('test.db')


In [51]:
# проверка
cursor = conn.execute(
    '''
    SELECT *
    FROM sales
    WHERE date > '2023-01-01'
    AND product = 'a'

 ''')

for row in cursor:
  print(row)

('2023-02-01', 'a', 0)
('2023-03-01', 'a', 4915)
('2023-04-01', 'a', 8284)
('2023-05-01', 'a', 5621)
('2023-06-01', 'a', 2764)
('2023-07-01', 'a', 3158)
('2023-08-01', 'a', 2453)
('2023-09-01', 'a', 5182)


# Побочные запросы

Запросы, направленные на проработку навыков написания SQL-запросов (в оригинальное задание не входило).

In [53]:
# Продажи продуктов в среднем по годам

cursor = conn.execute(
    '''
    SELECT product, strftime('%Y', DATETIME(date)) AS year, ROUND(AVG(sales),2) AS avg_sales
    FROM sales
    GROUP BY product, year
    ORDER BY product, year

 ''')

for row in cursor:
  print(row)

('a', '2022', 4765.0)
('a', '2023', 4648.56)
('b', '2022', 7063.25)
('b', '2023', 4951.33)
('c', '2022', 5592.55)
('c', '2023', 5879.89)


In [59]:
# Поиск месяца, в котором сумма продаж была максимальной в разбивке по каждому продукту и году

cursor = conn.execute(
    '''
    SELECT
          product,
          strftime('%Y', DATETIME(date)) AS year,
          strftime('%m', DATETIME(date)) AS month,
          MAX(sales) AS max_sales
    FROM sales
    GROUP BY product, year
    ORDER BY product, year, month

 ''')

for row in cursor:
  print(row)

('a', '2022', '10', 9399)
('a', '2023', '01', 9460)
('b', '2022', '07', 9920)
('b', '2023', '07', 9436)
('c', '2022', '01', 9879)
('c', '2023', '02', 9410)


In [46]:
# Прирост продаж по каждому продукту
# Поиск максимального и минимального прироста продаж в разрезе по каждому продукту


cursor = conn.execute(
    '''
    WITH lagged_table AS (
    SELECT
          product, sales, date,
          LAG(sales) OVER(PARTITION BY product ORDER BY date) AS lag_sales
    FROM sales
    )

    SELECT
          product,
          MIN(COALESCE(sales - lag_sales, 0)),
          MAX(COALESCE(sales - lag_sales, 0))
    FROM lagged_table
    GROUP BY product
    ORDER BY product, date

 ''')


for row in cursor:
  print(row)

('a', -9460, 7605)
('b', -7895, 8229)
('c', -8312, 6994)


In [60]:
# Соединение таблиц
# Поскольку таблица у меня одна, то создам ещё одну таблицу.

conn.execute(''' CREATE TABLE product_description(
                  product NVARCHAR,
                  description NVARCHAR); ''')
conn.commit()

print('The table product_description is created.')

The table product_description is created.


In [66]:
conn.execute('''
INSERT INTO
    product_description (product, description)
VALUES
    ('a','The first product'),
    ('b','The second product'),
    ('c','The third product');
''')

conn.commit()

In [71]:
# Объединение таблицы с суммарными продажами за всё время с таблицей, содержащей описание продуктов

cursor = conn.execute(
    '''
    SELECT
          product,
          SUM(sales) as sum_sales,
          description
    FROM sales
    LEFT JOIN product_description
    USING (product)
    GROUP BY product
    ORDER BY product

 ''')


for row in cursor:
  print(row)

('a', 99017, 'The first product')
('b', 129321, 'The second product')
('c', 114437, 'The third product')


# Основной запрос

Необходимо написать SQL-запрос, возращающий таблицу sales с добавленными столбцами:
- sales YTD - нарастающий итог продаж соответствующего product с начала года;
- sales MAT - суммарные продажи соответствующего product за последние 12 месяцев.


In [ ]:
cursor = conn.execute(
    '''
    SELECT
        date, product, sales,
        SUM(sales) OVER ( PARTITION BY strftime('%Y', DATETIME(date)), product  ORDER BY date) AS sales_ytd,
        SUM(sales) OVER ( PARTITION BY product ROWS BETWEEN 11 PRECEDING and CURRENT ROW) AS sales_mat
    FROM
        sales
    ORDER BY product, date
    ''')


for row in cursor:
  print(row)

('2022-01-01', 'a', 1979, 1979, 1979)
('2022-02-01', 'a', 6416, 8395, 8395)
('2022-03-01', 'a', 7332, 15727, 15727)
('2022-04-01', 'a', 3446, 19173, 19173)
('2022-05-01', 'a', 5276, 24449, 24449)
('2022-06-01', 'a', 5815, 30264, 30264)
('2022-07-01', 'a', 1070, 31334, 31334)
('2022-08-01', 'a', 8589, 39923, 39923)
('2022-09-01', 'a', 1794, 41717, 41717)
('2022-10-01', 'a', 9399, 51116, 51116)
('2022-11-01', 'a', 3988, 55104, 55104)
('2022-12-01', 'a', 2076, 57180, 57180)
('2023-01-01', 'a', 9460, 9460, 64661)
('2023-02-01', 'a', 0, 9460, 58245)
('2023-03-01', 'a', 4915, 14375, 55828)
('2023-04-01', 'a', 8284, 22659, 60666)
('2023-05-01', 'a', 5621, 28280, 61011)
('2023-06-01', 'a', 2764, 31044, 57960)
('2023-07-01', 'a', 3158, 34202, 60048)
('2023-08-01', 'a', 2453, 36655, 53912)
('2023-09-01', 'a', 5182, 41837, 57300)
('2022-01-01', 'b', 4195, 4195, 4195)
('2022-02-01', 'b', 9905, 14100, 14100)
('2022-03-01', 'b', 9628, 23728, 23728)
('2022-04-01', 'b', 4253, 27981, 27981)
('2022-05-0